In [ ]:
%run ./library

In [ ]:
tenant_id = dbutils.secrets.get(keyvault_name, 'tenant-id')
client_id_log= dbutils.secrets.get(keyvault_name, 'adls-client-id')
client_secret_log= dbutils.secrets.get(keyvault_name, 'adls-client-secret')
workspace_id = dbutils.widget.get('log_workspace_id')
env_name = dbutils.widget.get('env_name')

custom_log = dbutils.widget.get('log_analytics_custom_log_name')

water_mark_tab = dbutils.widget.get('delta_water_mark_table_name')
now = datetime.now(timezone.utc)
# Start time is one day before the current time
start_time = now - timedelta(days=30)
# End time is the current time
end_time = now
timespan = (start_time, end_time)

log_it = Log_fetch(tenant_id,client_id_log,client_secret_log,workspace_id,timespan)


In [ ]:
# getting max of uid value from the watermark table
max_last_uid = get_max_uid('custom_log_name',water_mark_tab,env_name)

# KQL - using which we want to extract data from the Log Analytics Custom Log
# We have constructed MetadataLogGuid column in all Log Analytics Custom Log tables
# It is in format of yyMMddhh000000001 - yyMMddhh999999999
query_base = f"""{custom_log}
| extend uid = todecimal(MetadataLogGuid)
| order by uid asc
| where uid>todecimal('{max_last_uid}')
"""
# Getting information about how many data is needed to be read and downloaded which is greater than max_last_uid value.
iteration_counter = log_it.fetch_log_gt_xuid_cnt(query_base)
divider = dbuitls.widget.get('number_of_records')
iterate = Math.ciel(iteration_counter/divider)

for i in range(iterate):
    max_last_uid = get_max_uid('custom_log_name',water_mark_tab,env_name)
    # we can add the where and project clause too to extract desired data and columns
    query_base = f"""{custom_log}
| extend uid = todecimal(MetadataLogGuid)
| order by uid asc
| where uid> todecimal('{max_last_uid}')
| take {divider}
"""
    response_data = log_it.fetch_logs(query_base)
    if response_data == 'Response Fail':
        print('Response Failed, Please check and try later')
        break
    else:
        for table in response_data:
            df = pd.Dataframe(data=table.rows,columns=table.columns)
    sparkDF = spark.createDataFrame(df)
    sparkDF.createOrReplaceTempView('df_data_view')
    data_max_uid_df = spark.sql('select max(uid) as uid_max from df_data_view')
    data_max_uid_df = data_max_uid_df.withColumn('log_name',F.lit(custom_log)).withColumn('env_name',F.lit(env_name))

    merge_into(data_max_uid_df,water_mark_tab,env_name)
    raw_path = f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{custom_log}/{env_name}'
    sparkDF.write.format('json').mode('append').save(raw_path)



